In [1]:
import json
import requests
import urllib.request
import pandas
import tarfile

In [2]:
r = requests.get('https://api.github.com/repos/spack/spack/releases')
#pandas.json_normalize(r, max_level=0)

fields = 'name', 'published_at', 'tarball_url'
df = pandas.DataFrame(map(lambda _: map(lambda x:_[x], fields), r.json()), columns=fields)[:21]
df.published_at=pandas.to_datetime(df.published_at)
df['release_date']=df.published_at.dt.date

In [3]:
import shutil, os, subprocess

numpkg=[]
for index, row in df.iterrows():
    if index>=0:
        ftpstream = urllib.request.urlopen(row['tarball_url'])
        with tarfile.open(fileobj=ftpstream, mode="r|gz") as thetarfile:
            prefix=os.path.join(os. getcwd(),thetarfile.firstmember.name)
            thetarfile.extractall()
            spack_init_script=os.path.join(prefix, 'share','spack','setup-env.sh')
#             print(os.path.isfile(spack_init_script))
#             cmd='{} && spack list | wc -l'.format(spack_init_script)
            process = subprocess.Popen('ls {} | wc -l'.format(os.path.join(prefix, 'var', 'spack', 'repos', 'builtin', 'packages')),stdout=subprocess.PIPE, shell=True)
            numpkg.append(int(process.communicate()[0].strip()))
#             print(numpkg[-1])
            shutil.rmtree(prefix, ignore_errors=True)
df['numpkg']=numpkg
df.head()

,name,published_at,tarball_url,release_date,numpkg
0,v0.16.1,2021-02-23 00:07:01+00:00,https://api.github.com/repos/spack/spack/tarba...,2021-02-23,5064
1,v0.16.0,2020-11-18 14:07:34+00:00,https://api.github.com/repos/spack/spack/tarba...,2020-11-18,5054
2,v0.15.4,2020-08-13 14:18:47+00:00,https://api.github.com/repos/spack/spack/tarba...,2020-08-13,4338
3,v0.15.3,2020-07-28 09:14:14+00:00,https://api.github.com/repos/spack/spack/tarba...,2020-07-28,4338
4,v0.15.2,2020-07-23 23:47:20+00:00,https://api.github.com/repos/spack/spack/tarba...,2020-07-23,4338


In [4]:
import altair as alt

chart = pandas.DataFrame({
    'date': df.published_at,
    'numpkg': df.numpkg,
    'release': df.name
})

line = alt.Chart(chart).mark_line(interpolate='step-after', color='gold').encode(
    x='date',
    y='numpkg'
)
points = alt.Chart(chart).mark_circle(
    size=50,
    color='steelblue'
).encode(
    x='date',
    y='numpkg',
    tooltip=['release', 'date', 'numpkg']
).interactive()

(line+points).interactive()

# # Create a selection that chooses the nearest point & selects based on x-value
# nearest = alt.selection(type='single', nearest=True, on='mouseover',
#                         fields=['x'], empty='none')
# selectors = alt.Chart(source).mark_point().encode(
#     x='date:Q',
#     opacity=alt.value(0),
# ).add_selection(
#     nearest
# )
# # Draw a rule at the location of the selection
# rules = alt.Chart(chart).mark_rule(color='gray').encode(
#     x='date:Q',
# ).transform_filter(
#     nearest
# )

# alt.layer(
#     line, selectors, points, rules
# ).properties(
#     width=600, height=300
# )

alt.LayerChart(...)

In [10]:
chart=(line+points).interactive()

# chart.properties(
#     width=800,
#     height=600
# )

# chart.configure_header(
#     labelFontSize=50
# )


#.save('spack_numpcg_vs_release.html')
chart.save('spack_numpcg_vs_release.html')

In [13]:
chart.save('spack_numpcg_vs_release.html', embed_options={'renderer':'svg'})